# Choropleth maps

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

In [ ]:
with open('../data/countries-simplified.json') as response:
    countries = json.load(response)

countries['features'][0]

In [ ]:
df = pd.read_csv("../data/fish-and-seafood-consumption-per-capita.csv")

df = df.rename({'Fish, Seafood- Food supply quantity (kg/capita/yr) (FAO, 2020)': 'consumption',
               'Entity': 'country'}, axis=1)

In [ ]:
range_color = (0, df.consumption.quantile(0.99))

In [ ]:
map_args = dict(
    geojson=countries,
    featureidkey='properties.ISO_A3',
    locations="Code",
    color="consumption", # lifeExp is a column of gapminder
    hover_name="country", # column to add to hover information
    animation_frame='Year',
    mapbox_style="carto-positron",
    color_continuous_scale='Plasma',
    range_color=range_color,
    height=600
)

In [ ]:
fig = px.choropleth_mapbox(df,
                           zoom=0,
                           **map_args
)

fig

In [ ]:
code_mapping = df[['country', 'Code']].drop_duplicates('Code').set_index('Code').to_dict('index')

df_cons_agg = df.groupby('Code')['consumption'].mean().reset_index()
df_cons_agg['country'] = df_cons_agg.apply(lambda x: code_mapping[x['Code']]['country'], axis=1)

df_cons_agg.sort_values('consumption', ascending=False).head(20)

In [ ]:
fig = px.choropleth_mapbox(df.query('country == "Maldives"'),
                           center={'lon': 73.5093, 'lat': 4.1755},
                           zoom=8,
                           **map_args
)

fig